In [ ]:
import csv
import datetime
import numpy
import os

from nupic.algorithms.sdr_classifier_factory import SDRClassifierFactory
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
from nupic.encoders.date import DateEncoder
from nupic.encoders.adaptive_scalar import AdaptiveScalarEncoder

from model_params import MODEL_PARAMS

_NUM_RECORDS = 3000

_INPUT_FILE_PATH = "gymdata.csv"

def runHotgym(numRecords):
    modelParams = MODEL_PARAMS["modelParams"]
    enParams = modelParams["sensorParams"]["encoders"]
    spParams = modelParams["spParams"]
    tmParams = modelParams["tmParams"]

    scalarEncoder = AdaptiveScalarEncoder(enParams["consumption"]["w"], n = enParams["consumption"]["n"])

    encodingWidth = scalarEncoder.getWidth()

    sp = SpatialPooler(
    inputDimensions=(encodingWidth),
    columnDimensions=(spParams["columnCount"]),
    potentialPct=spParams["potentialPct"],
    globalInhibition=spParams["globalInhibition"],
    #localAreaDensity=spParams["localAreaDensity"],
    numActiveColumnsPerInhArea=spParams["numActiveColumnsPerInhArea"],
    synPermInactiveDec=spParams["synPermInactiveDec"],
    synPermActiveInc=spParams["synPermActiveInc"],
    synPermConnected=spParams["synPermConnected"],
    boostStrength=spParams["boostStrength"],
    seed=spParams["seed"],
    wrapAround=False
  )

    tm = TemporalMemory(
    columnDimensions=(tmParams["columnCount"],),
    cellsPerColumn=tmParams["cellsPerColumn"],
    activationThreshold=tmParams["activationThreshold"],
    initialPermanence=tmParams["initialPerm"],
    connectedPermanence=spParams["synPermConnected"],
    minThreshold=tmParams["minThreshold"],
    maxNewSynapseCount=tmParams["newSynapseCount"],
    permanenceIncrement=tmParams["permanenceInc"],
    permanenceDecrement=tmParams["permanenceDec"],
    predictedSegmentDecrement=0.0,
    maxSegmentsPerCell=tmParams["maxSegmentsPerCell"],
    maxSynapsesPerSegment=tmParams["maxSynapsesPerSegment"],
    seed=tmParams["seed"]
  )

    classifier = SDRClassifierFactory.create()
    results = []
    with open(_INPUT_FILE_PATH, "r") as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()

        for count, record in enumerate(reader):
            if count >= numRecords: 
                print "Finished"
                break

            # Convert data string into Python date object.
            dateString = datetime.datetime.strptime(record[0], "%m/%d/%y %H:%M")
            # Convert data value string into float.
            consumption = float(record[1])

            # To encode, we need to provide zero-filled numpy arrays for the encoders
            # to populate.
            consumptionBits = numpy.zeros(scalarEncoder.getWidth())

            # Now we call the encoders create bit representations for each value.
            scalarEncoder.encodeIntoArray(consumption, consumptionBits)

            # Concatenate all these encodings into one large encoding for Spatial
            # Pooling.
            encoding = consumptionBits

            # Create an array to represent active columns, all initially zero. This
            # will be populated by the compute method below. It must have the same
            # dimensions as the Spatial Pooler.
            activeColumns = numpy.zeros(spParams["columnCount"])

            # Execute Spatial Pooling algorithm over input space.
            sp.compute(encoding, True, activeColumns)
            activeColumnIndices = numpy.nonzero(activeColumns)[0]

            # Execute Temporal Memory algorithm over active mini-columns.
            tm.compute(activeColumnIndices, learn=True)

            activeCells = tm.getActiveCells()

            # Get the bucket info for this input value for classification.
            bucketIdx = scalarEncoder.getBucketIndices(consumption)[0]

            # Run classifier to translate active cells back to scalar value.
            classifierResult = classifier.compute(
              recordNum=count,
              patternNZ=activeCells,
              classification={
              "bucketIdx": bucketIdx,
              "actValue": consumption
            },
            learn=True,
            infer=True
            )

            # Print the best prediction for 1 step out.
            oneStepConfidence, oneStep = sorted(
            zip(classifierResult[1], classifierResult["actualValues"]),
            reverse=True
            )[0]
            print("1-step: {:16} ({:4.4}%)".format(oneStep, oneStepConfidence * 100))
            results.append([oneStep, oneStepConfidence * 100, None, None])

    return results


if __name__ == "__main__":
    runHotgym(_NUM_RECORDS)

1-step:             21.2 (100.0%)
1-step:             21.2 (100.0%)
1-step:            19.76 (100.0%)
1-step:           15.242 (100.0%)
1-step:          12.0794 (100.0%)
1-step:          9.83558 (100.0%)
1-step:             47.5 (0.3968%)
1-step:             30.7 (0.402%)
1-step:             45.4 (0.8214%)
1-step:             46.1 (0.3968%)
1-step:             46.1 (0.48%)
1-step:             43.4 (0.5445%)
1-step:             46.1 (0.3968%)
1-step:             46.1 (0.3968%)
1-step:             36.6 (0.4419%)
1-step:             36.6 (0.6792%)
1-step:             35.7 (1.024%)
1-step:             35.7 (0.6316%)
1-step:             36.6 (1.379%)
1-step:             36.2 (0.6887%)
1-step:             46.1 (0.3968%)
1-step:             30.7 (0.4297%)
1-step:             30.7 (0.4297%)
1-step:              5.0 (0.4092%)
1-step:             46.1 (0.3968%)
1-step:             46.1 (0.3968%)
1-step:              5.0 (1.061%)
1-step:             5.93 (0.4277%)
1-step:             5.93 (0.4127